# Metody statystyczne i analityczne Big Data - EGZAMIN

Dane: "Informacje o pacjentach chorujących na raka piersi"

Cel: Porównanie 3 modeli i wybór najlepszego dla zbioru wdbc.csv zawierającego informacje o pacjentach chorujących na raka piersi - zamodelowanie zmiennej 'Diagnosis' - oznaczającej typ raka piersi (zmienna przyjmuje wartości: M = malignant, B = benign)

Opis zmiennych:
    1) ID number
    2) Diagnosis (M = malignant, B = benign)
    3) kolumny V1 - V30 - dane opisujące wymiary nowotworu

In [ ]:
#biblioteki:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.linear_model as skl_lm
import sklearn.ensemble

from sklearn import neighbors
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV


Wczytanie danych:

In [ ]:
df = pd.read_csv("wdbc.csv", index_col=0)

1. Wstępna eksploracyjna analiza danych

In [ ]:
df.head()


In [ ]:
df.describe().T

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.Diagnosis.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))

ax.pie(df.Diagnosis.value_counts(), labels = ["B", "M"], explode = (0, 0.05), shadow=True, radius=0.75,
       autopct = '%1.2f%%', startangle = 180, colors = ["#ff9999", "#66b3ff"])

ax.legend(['B - benign', 'M - malignant'])
ax.set_title("Diagnosis")
plt.show()

In [ ]:
df.isnull().any()

In [ ]:
df.columns

Wnioski:
    1. Zbiór danych posiada 31 kolumn oraz 569 wierszy, z czego tylko zmienna 'Diagnosis' jest kategoryczna.
    W następnej części zajmę się denotacją wartości zmiennej do wartości numerycznych oraz zbadaniem korelacji między zmiennymi.
    2. Niektóre z kolumn posiadają duży rozrzut pomiędzy średnią(mean), a połową wartości(50%). 
    W następnych krokach zbadam obecność punktów oddalonych, które mogą zaburzać rozkład normalny
    3. Zmienna 'Diagnosis' składa się z 62.74% B oraz 37.26% M 
    4. Zbiór danych nie posiada brakujących danych
    5. Nazwy kolumn posiadają w swoich nazwach znak ' ' - dla ułatwienia pracy nad zbiorem danych, w przyszłych krokach znaki te zostaną usunięte.
    

Denotacja wartości zmiennej 'Diagnosis':

In [ ]:
df['Diagnosis'] = df['Diagnosis'].map({'M': 1, 'B':0})

In [ ]:
df.Diagnosis

Ujednolicenie nazw kolumn:

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

Badanie korelacji między zmiennymi za pomocą współczynnika Pearsona:

In [ ]:
corr_P = df.corr("pearson")

In [ ]:
corr_P

In [ ]:
corr_P_tri = corr_P.where(np.triu(np.ones(corr_P.shape, dtype=bool),k=1)).stack().sort_values()

Filtrowanie wartości współczynnika korelacji na poziomie 0.8. Wartość 0.8 została dobrana aby zapewnić jak największa ilość danych użytych w póżniejszych krokach do modelowania.

In [ ]:
corr_P_tri[abs(corr_P_tri)> 0.80]

In [ ]:
sns.pairplot(df)

In [ ]:
plt.show()

Usunięcie kolumn, które posiadają współczynnik Pearsona powyżej 0.8:


In [ ]:
df = df.drop(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V11','V13', 'V14', 'V16', 'V21', 'V23', 'V26', 'V27', 'V27'], axis=1)

In [ ]:
df.head()

In [ ]:
corr_P2 = df.corr("pearson")

In [ ]:
corr_P2

In [ ]:
corr_P_tri2 = corr_P2.where(np.triu(np.ones(corr_P2.shape, dtype=bool),k=1)).stack().sort_values()

In [ ]:
corr_P_tri2[abs(corr_P_tri2)> 0.80]

Badanie obecności punktów oddalonych:


Metoda Z-score


In [ ]:
z = np.abs(stats.zscore(df))
print(z)

In [ ]:
threshold = 3
print(np.where(z > 3))

In [ ]:
df_out = df[(z < 3.5).all(axis=1)]

In [ ]:
df.shape

In [ ]:
df_out.shape

7.2% punktów oddalonych (zwiekszylam zakres z score ze wzgledu na bardzo duza ilosc punktow oddalonych - nie chcialam tracic tylu danych by przeciwdzialac przeuczeniu

In [ ]:
df.describe().T

In [ ]:
df_out.describe().T

Sprawdzenie dystrybucji poszczegolnych zmiennych

In [ ]:
X = df_out.drop(['Diagnosis'], axis=1)
X.hist(figsize=(20, 15), color = 'skyblue');

Model Nr1 - Regresja logistyczna:

Podział danych na zbiór treningowy oraz testowy (na poziomie 80% treningowy oraz 20% testowy):

In [ ]:
X = df_out.drop(['Diagnosis'], axis=1)
y = df_out['Diagnosis']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

Standaryzacja:

In [ ]:
m = X.mean()
s = X.std()

In [ ]:
X_train_std = (X_train- m)/s
X_test_std = (X_test - m)/s

Modelowanie:

In [ ]:
l_clf = LogisticRegression()
l_clf.fit(X_train_std,y_train)



In [ ]:
y_pred_train = l_clf.predict(X_train_std)

In [ ]:
y_pred = l_clf.predict(X_test_std)

Metoda walidująca model przez cztery wybrane metryki accuracy (ACC), precision (P), recall (R) i F1:

In [ ]:
def metrics(y_train, y_pred_train, y_test, y_pred):
    return {
        "ACC_train":  sklearn.metrics.accuracy_score(y_pred_train, y_train),
        "ACC_test": sklearn.metrics.accuracy_score(y_pred, y_test),
        "P_train":    sklearn.metrics.precision_score(y_pred_train, y_train),
        "P_test":   sklearn.metrics.precision_score(y_pred, y_test),
        "R_train":    sklearn.metrics.recall_score(y_pred_train, y_train),
        "R_test":   sklearn.metrics.recall_score(y_pred, y_test),
        "F1_train":   sklearn.metrics.f1_score(y_pred_train, y_train),
        "F1_test":  sklearn.metrics.f1_score(y_pred, y_test)
    }

Stworzenie ramki danych w celu zebrania wyników ze wszystkich wykonanych modeli:

In [ ]:
params = ["Log"]
res = [metrics(y_train, y_pred_train, y_test, y_pred)]
df_wyniki = pd.DataFrame(res, index=params)

In [ ]:
df_wyniki

Macierz pomyłek dla regresji logistycznej (zbiór treningowy oraz testowy):

In [ ]:
#zbiór treningowy
plot_confusion_matrix(l_clf, X_train_std, y_train)
plt.show()

In [ ]:
#zbiór testowy
plot_confusion_matrix(l_clf, X_test_std, y_test)
plt.show()

Model II - Metoda k-najbliższych sąsiadów

Modelowanie:

In [ ]:
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10)


Szukanie najlepszych hiperparametrów za pomocą metody Grid Search:

In [ ]:
knn_params = {'n_neighbors': range(1, 11), 'weights':['uniform', 'distance']}
knn_grid = GridSearchCV(knn, knn_params, cv=10, n_jobs=-1, scoring='recall')


In [ ]:
knn_grid.fit(X_train_std, y_train)

In [ ]:
knn_grid.best_params_, knn_grid.best_score_

In [ ]:
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')


In [ ]:
knn.fit(X_train_std, y_train)

In [ ]:
knn.get_params()

In [ ]:
y_pred_train_knn = knn.predict(X_train_std)

In [ ]:
y_pred_knn = knn.predict(X_test_std)

Metryki:

In [ ]:
params.append("knn")
res.append(metrics(y_train, y_pred_train_knn, y_test, y_pred_knn))
df_wyniki = pd.DataFrame(res, index=params)

In [ ]:
df_wyniki

Macierz pomyłek dla algorytmu k-najbliższych sąsiadów (zbiór treningowy oraz testowy):

In [ ]:
plot_confusion_matrix(knn, X_train_std, y_train)
plt.show()

In [ ]:
plot_confusion_matrix(knn, X_test_std, y_test)
plt.show()

Model III: Las losowy

Modelowanie:

Szukanie najlepszych hiperparametrów za pomocą metody Grid Search:

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17)

In [ ]:
rfc_params = {'max_features': range(1,11), 'min_samples_leaf': range(1,3), 'max_depth': range(3,13), 'criterion':['gini','entropy']}

rfc = RandomForestClassifier(n_estimators=100, random_state=17, n_jobs= -1)

gcv = GridSearchCV(rfc, rfc_params, n_jobs=-1, cv=skf, scoring='recall')

gcv.fit(X_train_std, y_train)

In [ ]:
gcv.best_params_, gcv.best_score_

In [ ]:
las = sklearn.ensemble.RandomForestClassifier(n_estimators=100, criterion ='gini', max_depth = 5, max_features = 6, min_samples_leaf = 2, random_state = 17, n_jobs=-1)
las.fit(X_train_std, y_train)

In [ ]:
y_pred_train_las = las.predict(X_train_std)

In [ ]:
y_pred_las = las.predict(X_test_std)

Metryki:

In [ ]:
params.append("las")
res.append(metrics(y_train, y_pred_train_las, y_test, y_pred_las))
df_wyniki = pd.DataFrame(res, index=params)

In [ ]:
df_wyniki

Las losowy umożliwia określenie 'ważności' zmiennych użytych do modelowania. 

In [ ]:
featimp = pd.Series(las.feature_importances_, index=X.columns[0:14]).sort_values(ascending=False)
print(featimp)

Macierz pomyłek dla lasu losowego (zbiór treningowy oraz testowy):

In [ ]:
plot_confusion_matrix(las, X_train_std, y_train)
plt.show()

In [ ]:
plot_confusion_matrix(las, X_test_std, y_test)
plt.show()

Wnioski: Porównanie metryk oraz wybranie najlepszego modelu

In [ ]:
df_wyniki

In [ ]:
df_train = df_wyniki.drop(['ACC_test', 'P_test', 'R_test', 'F1_test'], axis=1)

In [ ]:
df_test = df_wyniki.drop(['ACC_train', 'P_train', 'R_train', 'F1_train'], axis=1)

In [ ]:
ax = df_train.plot(kind='bar', figsize = (15,10), ylim = (0.85, 1.05), 
        color = ['gold', 'lightgreen', 'lightcoral', 'lightskyblue'],
        rot = 0, title ='Porównanie metryk dla zbioru uczącego',
        edgecolor = 'grey', alpha = 0.5)

plt.show()

In [ ]:
ax = df_test.plot(kind='bar', figsize = (15,10), ylim = (0.850, 1.05), 
        color = ['gold', 'lightgreen', 'lightcoral', 'lightskyblue'],
        rot = 0, title ='Porównanie metryk dla zbioru testującego',
        edgecolor = 'grey', alpha = 0.5)

plt.show()

Uzasadnienie wyboru modeli:

W powyższej analizie zastosowane zostały trzy modele: regresja logistyczna, metoda k-najbliższych sąsiadów oraz losowy las decyzjny.
Model regresji logistycznej znajduje zastosowanie, gdy zmienna jest dychotomiczna, to znaczy przyjmuje dwie wartości - co sprawia, że jest to pierwszy wybór po analizie powyższego zbioru danych, gdzie zmienna celu występuje jako dwie wartości M i B.
Algorytm k-najbliższych sąsiadów, w odróżnieniu od regresji, nie opiera się na trenowaniu modelu w celu generowania predykcji zmiennych objaśnianych. Z tego powodu nazywany jest również często algorytmem leniwym. Idea jego działania polega na przyporządkowaniu wszystkim danym wejściowym zestawu cech oraz umieszczeniu ich w wielowymiarowej przestrzeni w oparciu o miarę podobieństwa. Model został wybrany ze względu na szerokie zastosowanie w metodach opartych na klasyfikacji.
Ostatni model to losowy las decyzyjny. Został on wybrany ze względu na duże zastosowanie w metodach uczenia maszynowego dla klasyfikacji, czy regresji. Losowe lasy decyzyjne poprawiają tendencję drzew decyzyjnych do nadmiernego dopasowania się do zestawu treningowego.

Kryteria wyboru najlepszego modelu:

Macierz pomyłek jest metodą reprezentacji wyników predykcji w problemach klasyfikacji, dlatego też została zastosowana do ewaluacji powyższych modeli. Jest to tabela z czterema różnymi kombinacjami wartości przewidywanych i rzeczywistych. Macierz pomyłek przedstawia nie tylko błędy popełniane przez klasyfikator, ale co ważniejsze pokazuje jakie to są rodzaje błędów.

Accuracy - stosunek ilości poprawnie przewidzianych wartości do łącznej liczby wierszy w zbiorze testowym.

Precision - 'Jeżeli model przewidział, że wartość należy do danej klasy to jakie prawdopodobieństwo, że ta predykcja jest poprawna?'

Recall - odpowiada na pytanie: „Jakie jest prawdopodobieństwo, że model przewidzi iż wartość należy do klasy T, gdy faktycznie ta wartość do niej należy

F1-score - to średnia harmoniczna pomiędzy precyzją (precision) i czułością (recall)

# Podsumowanie:

Analizując powyższe tabelę oraz wykresy - zarówno dla zbioru uczącego jak i testującego - najlepszym modelem ze względu na wybrane metryki jest losowy las decyzyjny. Każda z metryk (dokładność, precyzyjność, czułość oraz F1-score) ma najwyższy wynik, zarówno analizując zbiór uczący jak i testujący. Ze względu na typ danych (klasyfikacja nowotworu jako złośliwy czy łagodny) dużą uwagę można zwrócić na metrykę precyzji - gdyż na jej wyniku nam najbardziej zależy - najwyższy wynik również odpowiada losowemu lasowi decyzyjnemu. Wybór  modelu pokrywa się z szerokim zastosowaniem lasu losowego, który jest niezwykle popularny w badaniach genetycznych, przewidywaniu aktywności biologicznej cząsteczek czy analizie dokumentów tekstowych - czyli tam, gdzie mamy do czynienia z dużą liczbą cech. Minusem lasów losowych może być fakt, że czasami się przeuczają. Podczas obróbki danych znakomita większość zmiennych została określona jako współliniowa (co jest całkowicie zrozumiałe, gdy wiemy iż zmienne odpowiadają wymiarom nowotworu) oraz ok. 7% wyników okreslona jako punkty odstające - przez co wybrane modele prowadziły do przeucznia. By temu zapobiec została zastosowana metoda Grid Search - w celu ustalenia optymalnych hiperparametrów dla danych modeli - dzięki temu zostały uzyskane zadowalające wyniki metryk. Razem z zastosowaniem techniki Grid Search - automatycznie wzrasta nam czas poświęcony na modelowanie, co może być wadą tego modelu - choć znów można podeprzeć się tu typem danych - dłuższy czas poświęcony na modelowanie może zwiększyć dokładność klasyfikacji, co przy odróżnieniu nowotworów jest bardzo istotne. 